# Gradiente del coste de un lote de datos

En este cuaderno, ampliamos el cálculo de la derivada parcial del notebook [*Gradiente de Regresión en un Punto*](https://github.com/joanby/matematicas-ml/blob/master/notebooks/single-point-regression-gradient.ipynb) para:

* Calcular el gradiente del error cuadrático medio en un lote de datos
* Visualizar el descenso de gradiente en acción

In [ ]:
import torch
import matplotlib.pyplot as plt

In [ ]:
xs=torch.tensor([0,1,2,3,4,5,6,7.])
ys=torch.tensor([1.86,1.31,.62,.33,.09,-.67,-1.23,-1.37])

In [ ]:
def regression(my_x,my_m,my_b):
  return my_m*my_x+my_b

In [ ]:
m=torch.tensor(0.9,requires_grad=True)
b=torch.tensor(0.1,requires_grad=True)

**Paso 1**: Paso hacia adelante

In [ ]:
yhats=regression(xs,m,b)
yhats

**Paso 2**: Comparamos $\hat{y}$ con el valor real de $y$ para caluclar el coste $C$

Como en el notebook [*Regresión en PyTorch*](https://github.com/joanby/matematicas-ml/blob/master/notebooks/regression-in-pytorch.ipynb), vamos a utilizar el error cuadrático medio, que promedia el coste cuadrático a través de múltiples puntos de datos: $$C = \frac{1}{n} \sum_{i=1}^n (\hat{y_i}-y_i)^2 $$

In [ ]:
def mse(my_yhat,my_y):
  sigma=torch.sum((my_yhat-my_y)**2)
  return sigma/len(my_y)

In [ ]:
C=mse(yhats,ys)
C

**Paso 3**: Utilizar autodiff para calcular el gradiente de $C$ en función de los parámetros

In [ ]:
C.backward()

In [ ]:
m.grad

In [ ]:
b.grad

**Volvemos a *Cálculo II* desliza aquí para derivar $\frac{\partial C}{\partial m}$ y $\frac{\partial C}{\partial b}$.**

$$ \frac{\partial C}{\partial m} = \frac{2}{n} \sum (\hat{y}_i - y_i) \cdot x_i $$

In [ ]:
2*1/len(ys)*torch.sum((yhats-ys)*xs)

In [ ]:
2*1/len(ys)*torch.sum(yhats-ys)

No necesitamos crear explícitamente un objeto $\nabla C$ independiente (delta griego invertido se llama *nabla* por «arpa» pero con gradiente es *del* como en «del C») para que el resto del código de este cuaderno se ejecute, pero vamos a crearlo por diversión ahora de todos modos y haremos uso de él en un cuaderno posterior relacionado:

In [ ]:
gradient=torch.tensor([[b.grad,m.grad]]).T
gradient

In [ ]:
def labeled_regresion_plot(my_x,my_y,my_m,my_b,my_C,include_grad=True):
  title='Coste={}'.format('%3g' % my_C.item())
  if include_grad:
    xlabel='m={}, m grad={}'.format('%3g' % my_m.item(),'%3g' % my_m.grad.item())
    ylabel='b={}, b grad={}'.format('%3g' % my_b.item(),'%3g' % my_b.grad.item())
  else:
    xlabel='m={}'.format('%3g' % my_m.item())
    ylabel='b={}'.format('%3g' % my_b.item())

  fig,ax=plt.subplots()
  plt.title(title)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)

  ax.scatter(my_x,my_y,zorder=3)

  x_min,x_max=ax.get_xlim()
  y_min=regression(x_min,my_m,my_b).detach().item()
  y_max=regression(x_max,my_m,my_b).detach().item()

  ax.set_xlim([x_min,x_max])
  ax.set_ylim([y_min,y_max])
  plt.grid(color='#dadada',linestyle='-')
  _=ax.plot([x_min,x_max],[y_min,y_max],c='C1')





In [ ]:
labeled_regresion_plot(xs,ys,m,b,C)

**Paso 4**: Gradiente descendente

$\frac{\partial C}{\partial m} = 36,3$ indica que un aumento de $m$ corresponde a un gran aumento de $C$.

Mientras tanto, $\frac{\partial C}{\partial b} = 6,26$ indica que un aumento de $b$ también corresponde a un aumento de $C$, aunque mucho menor que $m$.

Por lo tanto, en la primera ronda de entrenamiento, la fruta que cuelga más baja con respecto a la reducción del coste $C$ es disminuir la pendiente de la línea de regresión, $m$. También se producirá una disminución relativamente pequeña en la intersección $y$ de la línea, $b$.

In [ ]:
optimizer=torch.optim.SGD([m,b],lr=0.01)

In [ ]:
optimizer.step()

In [ ]:
C=mse(regression(xs,m,b),ys)
C

In [ ]:
labeled_regresion_plot(xs,ys,m,b,C,include_grad=False)

### Aclarar y repetir

Vamos a ver más rondas de entrenamiento:

In [ ]:
epochs=8
for epoch in range(epochs):
  optimizer.zero_grad() #reseteamos gradientes aceropara que no se acumilen
  yhats=regression(xs,m,b) #paso 1
  C=mse(yhats,ys) # paso2
  C.backward() #paso 3
  labeled_regresion_plot(xs,ys,m,b,C)
  optimizer.step() #paso 4



En rondas posteriores de entrenamiento, después de que la pendiente $m$ del modelo se haya acercado a la pendiente representada por los datos, $\frac{\partial C}{\partial b}$ se vuelve negativa, lo que indica una relación inversa entre $b$ y $C$. Mientras tanto, $\frac{\partial C}{\partial m}$ sigue siendo positivo.

Esta combinación dirige el descenso de gradiente para ajustar simultáneamente la intercepción $y$ $b$ hacia arriba y la pendiente $m$ hacia abajo con el fin de reducir el coste $C$ y, en última instancia, ajustar la línea de regresión perfectamente a los datos.